# Session configuration

> ⚠️ This is an advanced tutorial, make sure to learn the basics first.

When deploying a project, some additional configuration might be required.
As configuration often depends on where the project is deployed, the configuration can either be defined in YAML files or in Python code through the `atoti.config` package.

The configuration is passed as an argument of `create_session`, either as a path to the YAML file or directly as a Python object.

In [ ]:
import atoti as tt

## Port and URL

### Port

The port used by the atoti server defaults to a random available port but can be set to a specific one:

In [ ]:
config = tt.config.create_config(port=8080)

or in a YAML file:

```yaml
port: 8080
```

### URL pattern

This sets the URL returned by `session.url` to connect to the user interface.
The `{{host}}` and `{port}` placeholders will be replaced with, respectively, the actual host address and port number.
Defaults to `http://localhost:{port}`.

In a YAML file it can be set like that:

```yaml
url_pattern: https://example.com:{port}/
```

equivalent to:

In [ ]:
config = tt.config.create_config(url_pattern="https://example.com:{port}/")

## Metadata database

Some data, such as the dashboards generated by the users, is not part of the data sources but is stored anyway in what we call the metadata database.
By default, this database is in memory so everything is lost when the atoti session is closed.
However, it can also be persisted to a file.

This is an example of configuration that stores the metadata DB in a file:

```yaml
metadata_db: ./metadata.db
```

which is equivalent to the following Python code:

In [ ]:
config = tt.config.create_config(metadata_db="./metadata.db")

## Security

> **Atoti+ only**: security configuration is only available in Atoti+.

When sharing your application with other users, you can set up security to configure which users are allowed to connect to the application and which part of the data they are allowed to see.

### Roles

Roles are a way to restrict what users can see in a cube.
By default, all users mentioned in your configuration file have a role called `ROLE_USER` which gives access to the full cube.
You can define additional roles with restrictions and give these roles to your users.

This in an example of roles in a YAML file:

```yaml
roles:
  - name: ROLE_FRANCE
    restrictions:
      Country: France
      Currency: Euro
  - name: ROLE_AMERICA
    restrictions:
      Country: [USA, Canada]
  - name: ROLE_CHINA
    restrictions:
      Country: [China]
```

Roles can also be defined directly in Python:

In [ ]:
french = tt.config.create_role(
    "ROLE_FRANCE", restrictions={"Country": "France", "Currency": "Euro"}
)
american = tt.config.create_role(
    "ROLE_AMERICA", restrictions={"Country": ["USA", "Canada"]}
)
chinese = tt.config.create_role("ROLE_CHINA", restrictions={"Country": "China"})

config = tt.config.create_config(roles=[french, american, chinese])

Roles are defined per column.
Column restrictions are inherited by all hierarchies based on this column.
For instance, in the previous configuration, a user with the role `ROLE_AMERICA` will only see the data related to USA and Canada and won't see the data for France.

#### Combining roles

Restrictions on different hierarchies are intersected.
For instance, in the previous configuration, a user with the role `ROLE_FRANCE` will only see the data where the country is France AND the currency is Euro.

However, if a user has several roles with restrictions on the same hierarchies, the access to the union of restricted members will be granted.
For instance, in the previous configuration, a user with both `ROLE_AMERICA` and `ROLE_CHINA` will see the data where the country is USA, Canada, OR China.

### Authentication

#### Basic

Basic authentication only requires usernames and passwords.
It is the easiest way to get started with security on a project, you only have to define the users, their password, and their roles.

This is an example of YAML configuration defining 3 users:

```yaml
authentication:
  basic:
    users:
      - name: admin
        password: nidma
        roles:
          - ROLE_ADMIN
      - name: user1
        password: 1resu
        roles:
          - ROLE_FRANCE
      - name: user2
        password: 2resu
        roles:
          - ROLE_UK
```

Which is equivalent to the following Python code:

In [ ]:
admin = tt.config.create_basic_user("admin", "nidma", roles=["ROLE_ADMIN"])
user1 = tt.config.create_basic_user("user1", "1resu", roles=["ROLE_FRANCE"])
user2 = tt.config.create_basic_user("user2", "2resu", roles=["ROLE_UK"])
basic = tt.config.create_basic_authentication(
    [admin, user1, user2], realm="Configuration Tutorial"
)

config = tt.config.create_config(authentication=basic)

#### OpenID Connect

atoti is compliant with any OpenID Connect OAuth2 authentication provider (Auth0, Google, Keycloak, etc.).

The configuration requires:

- information about your application (clientId, clientSecret)
- information about the OAuth2 provider you are using
- _optional_: a role mapping between the OAuth2 roles/users and atoti roles

This is an example YAML configuration for OAuth2 giving access to other users:

```yaml
authentication:
  oidc:
    provider_id: myProvider # This configures redirectUrls, the format of the redirect URL is {baseUrl}/login/oauth2/code/{providerId}
    issuer_url: myIssuer # The URL of the authentication server
    client_id: clientId
    client_secret: clientSecret
    name_attribute: email # The value to use as the displayed username in the application
    scopes: # The scopes to request access to (openid is passed by default)
      - email
      - profile
    paths_to_authorities:
      - paths/to/authorities # The path to the authorities in the claims contained in the token
    role_mapping:
      ROLE_ADMIN:
        - ROLE_ADMIN # map a role granted by the OAuth2 server to an atoti role
      test_admin:
        - ROLE_ADMINd # map atoti roles to a username
```

In [ ]:
oidc = tt.config.create_oidc_authentication(
    provider_id="MyProvider",
    issuer_url="MyIssuer",
    client_id="clientId",
    client_secret="clientSecret",
    scopes=["email", "profile"],
    name_attribute="username",
    paths_to_authorities=["path/to/authorities"],
    role_mapping={"ROLE_ADMIN": ["ROLE_ADMIN", "ROLE_TEST"]},
)

## Sampling mode

This sets the default sampling mode for all the stores.
When building the data model, it is more efficient to work only on a subset of the data.
Once the modeling is over, everything can be loaded with `Session.load_all_data()`.

This mode will load only the first 10,000 lines for each store:

```yaml
sampling_mode:
  first_lines: 10000
```

This mode will load only the first file for each store:

```yaml
sampling_mode:
  first_files: 1
```

This mode will load all the data:

```yaml
sampling_mode: full
```

These can also be defined in Python:

In [ ]:
config = tt.config.create_config(sampling_mode=tt.sampling.first_lines(10000))
config = tt.config.create_config(sampling_mode=tt.sampling.first_files(1))
config = tt.config.create_config(sampling_mode=tt.sampling.FULL)

## Application memory

atoti loads all the data in memory. The max memory can be set to increase the capacity of the application.

The format is a string containing a number followed by a unit among G, M and K, for instance "64G".
This actually sets the -Xmx JVM parameters and defaults to the JVM default memory which is 25% of the machine memory.

```yaml
max_memory: 64G
```

In [ ]:
config = tt.config.create_config(max_memory="64G")

## Advanced Java Arguments

It is possible to provide additional arguments to Java in order to do some custom optimization or debugging

In YAML it is done with a list of arguments:

```yaml
java_args:
  - "-Xms1g"
  - "-XX:+UseG1GC"
```

equivalent to this Python code:

In [ ]:
config = tt.config.create_config(java_args=["-Xms1g", "-XX:+UseG1GC"])

## Configuration inheritance

Sometimes, you want to use the same base configuration for all your projects. For instance, if you are using the Auth0 account of your company, you won't want to repeat its properties everywhere. To avoid that, a configuration file can inherit the default configuration file and be merged with it.

A default configuration file can be placed at `$ATOTI_HOME/config.yml` where `$ATOTI_HOME` defaults to `$HOME/.atoti`.

By default, the configuration will inherit the default one, if you don't want this behaviour, add `inherit: False` to your configuration.

A classic example is to have a default `config.yml` with the Auth0 properties:

```yaml
authentication:
  auth0:
    issuer: myIssuer
    audience: myAudience
    client_id: myClientId
    client_secret: myClientSecret
```

and have a configuration specific to each project with the role mapping:

```yaml
authentication:
  auth0:
    roles:
      admin:
        - ROLE_ADMIN
      french_user:
        - ROLE_FRANCE
```

## Environment variables

Environment variables can be used in the configuration with the following syntax:

```yaml
authentication:
  basic:
    users:
      - name: ${{ env.PROJECT_USERNAME }}
        password: ${{ env.PROJECT_PASSWORD }}
```